In [70]:
import numpy as np
import pandas as pd
from scipy.stats.mstats import winsorize
from statsmodels.formula.api import ols
from tabulate import tabulate

In [322]:
def load_material_price():
    df = pd.read_excel('data/Construction_material_market_prices(1999-2024).xlsx', 
                      sheet_name='Price')
    
    used_cols = []
    cols = list(df.columns)
    
    for col in cols:
        if 'Data Series' not in col:
            used_cols.append(col)
            
    df = df.melt(id_vars=['Data Series'],
    value_vars=used_cols,
    var_name='Time', value_name='Price')
    
    df['year'] = df['Time'].apply(lambda x: int(x.split()[0]))
    df = df.groupby(['Data Series', 'year']).Price.mean().reset_index()
    df = df.rename(columns={'Data Series': 'material'})
    
    df = pd.pivot_table(df, values = 'Price', index=['year'], columns = 'material').reset_index()
    
    df = df.rename(columns={'Cement In Bulk (Ordinary Portland Cement) (Dollar Per Tonne)': 'cement',
                           'Concreting Sand (Dollar Per Tonne)': 'concreting_sand',
                           'Granite (20mm Aggregate) (Dollar Per Tonne)': 'granite',
                           'Ready Mixed Concrete (Dollar Per Cubic Metre)': 'mixed_concrete',
                           'Steel Reinforcement Bars (16-32mm High Tensile) (Dollar Per Tonne)': 'steel'})
    return df

In [323]:
def load_hdb_resale_price():
    used_columns = ['year','town', 'covid','flat_type', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
    df1 = pd.read_csv('data/ResaleFlatPricesBasedonRegistrationDateFromJan2015toDec2016.csv')
    df2 = pd.read_csv('data/ResaleflatpricesbasedonregistrationdatefromJan2017onwards.csv')
    df = pd.concat([df1, df2])
    df['year'] = df['month'].apply(lambda x: int(x.split('-')[0]))
    df['month'] = df['month'].apply(lambda x: int(x.split('-')[-1]))
    df['flat_type'] = df['flat_type'].apply(lambda x: x.lower())
    df['town'] = df['town'].apply(lambda x: x.lower())
    df['covid'] = df.apply(lambda x: 1 if x['year']>=2020 and x['month']>4 else 0, axis=1)
    return df[used_columns]

def load_hdb_rental_price():
    hdb_rental_price = pd.read_excel('data/HDB Rental Price (from 2007-3qtr to 2023-3qtr).xlsx')
    hdb_rental_price.columns = hdb_rental_price.iloc[0]
    hdb_rental_price = hdb_rental_price[1:]
    del hdb_rental_price[hdb_rental_price.columns[0]]
      
    hdb_rental_price = hdb_rental_price.melt(
    id_vars=['Town', 'Year '],
    value_vars=['1-Room', '2-Room', '3-Room', '4-Room','5-Room', 'Executive'],
    var_name='Room Type', value_name='Price')
    
    hdb_rental_price = hdb_rental_price[(hdb_rental_price['Price'] != '-') & (hdb_rental_price['Price'] != '*')]
    hdb_rental_price['Room Type'] = hdb_rental_price['Room Type'].apply(lambda x: x.replace('-', ' ').lower())
    hdb_rental_price = hdb_rental_price.rename(columns={'Year ': 'Year'})
    hdb_rental_price['Price'] = hdb_rental_price['Price'].apply(lambda x: int(x.replace('$', '').replace('.','').replace(',','')))
    hdb_rental_price['Town'] = hdb_rental_price['Town'].apply(lambda x: x.lower())
    
    df = hdb_rental_price.groupby(['Town','Year', 'Room Type']).mean()
    df = df.reset_index()
    
    return df[df['Year']>=2015]

def combine_rental_and_resale_price():
#   central defined by MRT station. If it could reach central area within 3 stops, we considered it central area
    central = ['bukit merah', 'geylang', 'queenstown', 'toa payoh']   
    
    resale = load_hdb_resale_price()
    rental = load_hdb_rental_price()
    
    combined_df = resale.merge(rental, left_on=['year', 'town', 'flat_type'], right_on=['Year', 'Town', 'Room Type'])
    
    used_columns=list(resale.columns)
    used_columns.append('rental_price')
    combined_df = combined_df.rename(columns={'Price': 'rental_price'})
    combined_df = combined_df[used_columns]
    
    combined_df['central_marker'] = combined_df['town'].apply(lambda x: 1 if x in central else 0)
    return combined_df

In [324]:
hdb_resale_price = load_hdb_resale_price()
hdb_rental_price = load_hdb_rental_price()
material_price = load_material_price()
df = combine_rental_and_resale_price()

# combine with material price
df = df.merge(material_price, left_on=['year'], right_on=['year'])

In [343]:
# res = ols('resale_price ~ covid + town + flat_type + floor_area_sqm + cement + concreting_sand + granite', data=df).fit()
res = ols('resale_price ~ covid + central_marker + flat_type + rental_price + cement + concreting_sand + granite + steel', data=df).fit()
with open('summary_all.txt', 'w') as fh:
    fh.write(res.summary().as_text())

In [344]:
df_central = df[df['central_marker']==1]
res = ols('resale_price ~ covid + central_marker + flat_type + rental_price + cement + concreting_sand + granite + steel', data=df_central).fit()
with open('summary_central.txt', 'w') as fh:
    fh.write(res.summary().as_text())

In [345]:
df_non_central = df[df['central_marker']==0]
res = ols('resale_price ~ covid + central_marker + flat_type + rental_price + cement + concreting_sand + granite + steel', data=df_non_central).fit()
with open('summary_non_central.txt', 'w') as fh:
    fh.write(res.summary().as_text())